In [ ]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import networkx as nx
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import pyspace
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
from importlib import reload
pd.set_option('display.max_rows', 500)
reload(pyspace)
import seaborn as sns
from esda.moran import Moran
# sns.set_theme(font = 'Helvetica')
%matplotlib inline
from numba import NumbaDeprecationWarning
import pyogrio
import warnings
import esda
# Suppress NumbaDeprecationWarning
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)

In [ ]:
from matplotlib.patheffects import withStroke

In [ ]:
reload(pyspace)

In [ ]:
main_folder = Path('../Manuscript/Economic analysis of integrative medicine/')
data_folder = Path('../Data')
results_folder = main_folder/'Results'
figures_folder = main_folder/'Figures'

In [ ]:
# data = pd.read_parquet(data_folder/'processed'/'full_dataset_nonull_nominors.parquet.gzip')

In [ ]:
data = pd.read_parquet(data_folder/'processed'/'df_treated_filtered_nominors.parquet.gzip')
data = gpd.GeoDataFrame(data, crs = 4326, geometry=gpd.points_from_xy(data.lon_masked, data.lat_masked))

data = data.to_crs(2056)
data['E'], data['N'] = data['geometry'].x, data['geometry'].y

data_final = data[data.treatment.isnull()==False]

In [ ]:
# df_aos_costs = pd.read_csv('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/processed/df_aos_costs.csv')
# df_aos_costs = gpd.GeoDataFrame(df_aos_costs, crs = 2056, geometry=gpd.points_from_xy(df_aos_costs.E, df_aos_costs.N))


# df_lca_costs = pd.read_csv('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/processed/df_lca_costs.csv')
# df_lca_costs = gpd.GeoDataFrame(df_lca_costs, crs = 2056, geometry=gpd.points_from_xy(df_lca_costs.E, df_lca_costs.N))

# df_cam_costs = pd.read_csv('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/processed/df_cam_costs.csv')
# df_cam_costs = gpd.GeoDataFrame(df_cam_costs, crs = 2056, geometry=gpd.points_from_xy(df_cam_costs.E, df_cam_costs.N))


# df_lca_costs_t1 = pd.read_csv('/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/processed/df_lca_costs_t1.csv')
# df_lca_costs_t1 = gpd.GeoDataFrame(df_lca_costs_t1, crs = 2056, geometry=gpd.points_from_xy(df_lca_costs_t1.E, df_lca_costs_t1.N))

In [ ]:
# del data

In [ ]:
data_2017 = data_final[data_final.NOANNEE == 2017]
data_2018 = data_final[data_final.NOANNEE == 2018]
data_2019 = data_final[data_final.NOANNEE == 2019]
data_2020 = data_final[data_final.NOANNEE == 2020]
data_2021 = data_final[data_final.NOANNEE == 2021]

data_2017 = gpd.GeoDataFrame(data_2017, crs = 2056, geometry = data_2017['geometry'] )
data_2018 = gpd.GeoDataFrame(data_2018, crs = 2056, geometry = data_2018['geometry'] )
data_2019 = gpd.GeoDataFrame(data_2019, crs = 2056, geometry = data_2019['geometry'] )
data_2020 = gpd.GeoDataFrame(data_2020, crs = 2056, geometry = data_2020['geometry'] )
data_2021 = gpd.GeoDataFrame(data_2021, crs = 2056, geometry = data_2021['geometry'] )

In [ ]:
cantons = gpd.read_file(
    '/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp')
communes = gpd.read_file(
    '/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.shp')

# Spatial analyses
## Data as individual points
### Global autocorrelation of main features

- AOS Yearly Spending
- LCA Yearly Spending
- CDS

In [ ]:
individual_results_folder = results_folder/'Individual'

In [ ]:
globalautocorr_result_folder = individual_results_folder/'Global Autocorrelation'
if not os.path.exists(globalautocorr_result_folder):
    os.makedirs(globalautocorr_result_folder)

In [ ]:
def GlobalMoranI(db, col, year, distance, w, result_folder, seed=12345):
    xlabel = f"Global Moran's I - {col} - {year} - {distance}NN"
    file_path = result_folder / f'{xlabel}.pdf'

    # Check if the file already exists
    if file_path.exists():
        print(f"File '{file_path}' already exists. Skipping execution.")
        return None
    
    # Compute Moran's I
    y = db[col]
    np.random.seed(seed)
    mi = esda.moran.Moran(y, w)
    print(col, year, distance, mi.I, mi.p_sim, mi.z_sim)
    
    # Moran's I plot
    sns.kdeplot(mi.sim, fill=True)
    plt.vlines(mi.I, 0, plt.ylim()[1]*0.1, color='r', label='Moran\'s I')
    plt.vlines(mi.EI, 0, plt.ylim()[1]*0.1, label='Expected I')
    plt.xlabel(xlabel)
    plt.ylabel('Density')
    plt.title('Moran\'s I Distribution')
    plt.legend()
    # Save figure
    plt.savefig(file_path, dpi=320, bbox_inches='tight')
    plt.close()  # Close the plot after saving

    return mi

In [ ]:
col_names = ['PRESTATIONS_BRUTES_AOS', 'PRESTATIONS_BRUTES_LCA', 'cds']  # replace with your actual column names

# Calculate weights once for each year and store them
years = [2017, 2018, 2019, 2020, 2021]
weights_by_year = {}
nn = 32

for year, df in zip(years, [data_2017, data_2018, data_2019, data_2020, data_2021]):
    # Calculate weights here (w)
    # Store the weights in the dictionary
    if year not in weights_by_year.keys():
        weights_by_year[year] = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)


# Now iterate over each column and use the pre-calculated weights
for col_name in col_names:
    for year, df in zip(years, [data_2017, data_2018, data_2019, data_2020, data_2021]):
        w = weights_by_year[year]  # Retrieve pre-calculated weights
        mi = GlobalMoranI(db=df, col=col_name, year=year, distance=nn, w=w, result_folder=globalautocorr_result_folder)

### Local autocorrelation using Getis Ord Gi* statistic

In [ ]:
localautocorr_result_folder = individual_results_folder/'Local Autocorrelation'
if not os.path.exists(localautocorr_result_folder):
    os.makedirs(localautocorr_result_folder)

In [ ]:
# Now iterate over each column and use the pre-calculated weights
for col_name in col_names:
    for year, df in zip(years, [data_2017, data_2018, data_2019, data_2020, data_2021]):
        w = weights_by_year[year]  # Retrieve pre-calculated weights
        getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
        fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.01, markersize_l=0.1, p_001=False, commune_name=False)
        xlabel = f"Getis - {col_name} - {year} - {nn}NN"
        file_path = localautocorr_result_folder / f'{xlabel}.png'
        plt.savefig(file_path, dpi=1000, bbox_inches='tight')

In [ ]:
dict_labels = {'PRESTATIONS_BRUTES_AOS':'Conv. Med. Yearly Spending (CHF)',
              'PRESTATIONS_BRUTES_LCA':'Compl. Med. Yearly Spending (CHF)',
              'cds':'Chronic Disease Score (CDS)',
              'ihs_cost_lca':'IHS transformed CAM claims amount'}

In [ ]:
df_aos_costs = data_final[data_final.PRESTATIONS_BRUTES_AOS > data_final.MTFRANCHISECOUV]
df_lca_costs = data_final[data_final.PRESTATIONS_BRUTES_LCA > 0]
df_cam_costs = data_final[data_final.PRESTATIONS_BRUTES_CAM > 0]

### AOS USE

In [ ]:
avg_prestation_aos_by_canton = df_aos_costs.groupby('CANTON_NAME', observed=True)['PRESTATIONS_BRUTES_AOS'].median()
rank = avg_prestation_aos_by_canton.argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638
rank = rank.sort_values().index.tolist()
pal = sns.color_palette("Greens", len(avg_prestation_aos_by_canton))
fig, ax = plt.subplots(figsize = (8,8))
sns.boxplot(data=df_aos_costs, y="PRESTATIONS_BRUTES_AOS",x = 'CANTON_NAME',hue = 'lang_region',showfliers = False,dodge = False, order = rank, ax = ax)
# ax.set_ylim(0,5000)
plt.xticks(rotation = 90)
plt.legend(title='')
sns.despine()
plt.grid(axis = 'y')
plt.xlabel('Cantons', fontsize=12)
plt.ylabel('Annual CM (MHI) expenditures (CHF)', fontsize=12)
plt.savefig(results_folder/'Figures_rapport/Avg_CM_MHI_by_canton.png', dpi = 300, bbox_inches='tight')

In [ ]:
gdf_names = communes[communes.NAME.isin(['Lausanne','Genève','Zürich','Basel','Bern'])]

In [ ]:
gdf_names = gdf_names[gdf_names.EINWOHNERZ.isnull()==False]

In [ ]:
col_name = 'ihs_cost_aos'
nn=32
df = df_aos_costs[~df_aos_costs[col_name].isnull()]

w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
for x, y, label in zip(gdf_names.geometry.centroid.x, gdf_names.geometry.centroid.y, gdf_names['NAME']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom',
            path_effects=[withStroke(linewidth=3, foreground='white')], zorder=8)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=640, bbox_inches='tight')

In [ ]:
reload(pyspace)

In [ ]:
xlabel = f"Getis Bar Plot - {col_name} - {nn}NN"
file_path = Path(localautocorr_result_folder) / f'{xlabel}.png'
fig, ax = pyspace.plot_getis_by_class(df = df,x = f'{col_name}_G_cl',y = 'PRESTATIONS_BRUTES_AOS', label = 'Annual CM (MHI) expenditures (CHF)', xtick_size=8, title_size=12, xlabel_size=8,ylabel_size= 8, p_001=False, showfliers = False)
plt.savefig(file_path, dpi=320, bbox_inches='tight')

### CAM - SI USE

In [ ]:
avg_prestation_aos_by_canton = df_lca_costs.groupby('CANTON_NAME', observed=True)['PRESTATIONS_BRUTES_LCA'].median()
rank = avg_prestation_aos_by_canton.argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638
rank = rank.sort_values().index.tolist()
pal = sns.color_palette("Greens", len(avg_prestation_aos_by_canton))
fig, ax = plt.subplots(figsize = (8,8))
sns.boxplot(data=df_lca_costs, y="PRESTATIONS_BRUTES_LCA",x = 'CANTON_NAME',hue = 'lang_region',showfliers = False,dodge = False, order = rank, ax = ax)
# ax.set_ylim(0,5000)
plt.xticks(rotation = 90)
plt.legend(title='Language regions')
sns.despine()
plt.grid(axis = 'y')
plt.xlabel('Cantons', fontsize=12)
plt.ylabel('Annual CAM (SI) expenditures (CHF)', fontsize=12)
plt.savefig(results_folder/'Figures_rapport/Avg_CAM_SI_by_canton.png', dpi = 300, bbox_inches='tight')

### CAM - MHI USE

In [ ]:
avg_prestation_aos_by_canton = df_cam_costs.groupby('CANTON_NAME', observed=True)['PRESTATIONS_BRUTES_CAM'].median()
rank = avg_prestation_aos_by_canton.argsort().argsort()   # http://stackoverflow.com/a/6266510/1628638
rank = rank.sort_values().index.tolist()
pal = sns.color_palette("Greens", len(avg_prestation_aos_by_canton))
fig, ax = plt.subplots(figsize = (8,8))
sns.boxplot(data=df_cam_costs, y="PRESTATIONS_BRUTES_CAM",x = 'CANTON_NAME',hue = 'lang_region',showfliers = False,dodge = False, order = rank, ax = ax)
# ax.set_ylim(0,5000)
plt.xticks(rotation = 90)
plt.legend(title='Language regions')
sns.despine()
plt.grid(axis = 'y')
plt.xlabel('Cantons', fontsize=12)
plt.ylabel('Annual CAM (MHI) expenditures (CHF)', fontsize=12)
plt.savefig(results_folder/'Figures_rapport/Avg_CAM_MHI_by_canton.png', dpi = 300, bbox_inches='tight')

In [ ]:
reload(pyspace)

In [ ]:
col_name = 'ihs_cost_cam'
nn=32
df = df_cam_costs[~df_cam_costs[col_name].isnull()]

w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
for x, y, label in zip(gdf_names.geometry.centroid.x, gdf_names.geometry.centroid.y, gdf_names['NAME']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom',
            path_effects=[withStroke(linewidth=3, foreground='white')], zorder=8)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=640, bbox_inches='tight')

In [ ]:
xlabel = f"Getis Bar Plot - {col_name} - {nn}NN"
file_path = Path(localautocorr_result_folder) / f'{xlabel}.png'
fig, ax = pyspace.plot_getis_by_class(df = df,x = f'{col_name}_G_cl',y = 'PRESTATIONS_BRUTES_CAM', label = 'Annual CAM (MHI) expenditures (CHF)', xtick_size=8, title_size=12, xlabel_size=8,ylabel_size= 8, p_001=False, showfliers = False)
plt.savefig(file_path, dpi=320, bbox_inches='tight')

In [ ]:
col_name = 'model0cam_continuous_res'
nn=32
df = df_cam_costs[~df_cam_costs[col_name].isnull()]
w = lps.weights.KNN(cKDTree(get_points_array(df_cam_costs.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=1000, bbox_inches='tight')

In [ ]:
col_name = 'model5cam_continuous_res'
nn=32
df = df_cam_costs[~df_cam_costs[col_name].isnull()]
w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=1000, bbox_inches='tight')

## LCA USE

In [ ]:
col_name = 'ihs_cost_lca'
nn=32
df = df_lca_costs[~df_lca_costs[col_name].isnull()]

w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)

for x, y, label in zip(gdf_names.geometry.centroid.x, gdf_names.geometry.centroid.y, gdf_names['NAME']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom',
            path_effects=[withStroke(linewidth=3, foreground='white')], zorder=8)
# ax.set_title('A', loc = 'left', size= 16)
ax.set_axis_off()  # Hide axes

xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=640, bbox_inches='tight')

In [ ]:
xlabel = f"Getis Bar Plot - {col_name} - {nn}NN"
file_path = Path(localautocorr_result_folder) / f'{xlabel}.png'

fig, ax = pyspace.plot_getis_by_class(df = df,x = f'{col_name}_G_cl',y = 'PRESTATIONS_BRUTES_LCA', label = 'Annual CAM (SI) expenditures (CHF)', xtick_size=8, title_size=12, xlabel_size=8,ylabel_size= 8, p_001=False, showfliers = False)
plt.savefig(file_path, dpi=320, bbox_inches='tight')

In [ ]:
col_name = 'model0lca_continuous_res'
nn=32
# w = lps.weights.KNN(cKDTree(get_points_array(df_aos_costs.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=640, bbox_inches='tight')

In [ ]:
col_name = 'model3lca_continuous_res'
nn=32
df = df_lca_costs[~df_lca_costs[col_name].isnull()]
w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=640, bbox_inches='tight')

In [ ]:
col_name = 'model5lca_continuous_res'
nn=32
df = df_lca_costs[~df_lca_costs[col_name].isnull()]
w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
getis_values = pyspace.compute_getis(df, col_name, w, 999, transform_type='B', p_001=False)
fig, ax = pyspace.plotGetisMap(df, f"{col_name}_G_cl", markersize_s=0.08, markersize_l=1, p_001=False, commune_name=False)
xlabel = f"Getis - {col_name} - {nn}NN"
file_path = localautocorr_result_folder / f'{xlabel}.png'
plt.savefig(file_path, dpi=640, bbox_inches='tight')

## Data as polygons
### Create hexbins for aggregated evaluation of spatial statistics

In [ ]:
from shapely.geometry import Polygon

# Define the bounding box coordinates (xmin, ymin, xmax, ymax)
xmin, ymin, xmax, ymax = data_final.total_bounds

# Set the hexagon dimensions (dx is the distance from center to vertex)
dx = 500  # in meters

# Calculate vertical distance (dy)
dy = dx * 3 ** 0.5

# Initialize an empty list to hold hexagon polygons
hexagon_list = []

# Calculate the number of hexagons that fit in x and y dimensions
nx = int((xmax - xmin) / (3/2 * dx))
ny = int((ymax - ymin) / dy)

# Generate hexagons
for i in range(nx):
    for j in range(ny):
        x = xmin + i * 3/2 * dx
        y = ymin + j * dy

        # Offset every other row
        if i % 2 == 1:
            y += dy / 2

        # Create vertices of the hexagon
        vertices = [(x + dx * np.cos(np.pi / 3 * t), y + dx * np.sin(np.pi / 3 * t)) for t in range(6)]
        
        # Create a polygon from vertices and add to list
        hexagon = Polygon(vertices)
        hexagon_list.append(hexagon)

# Create a GeoDataFrame from the hexagon polygons
h3_500_gdf = gpd.GeoDataFrame(geometry=hexagon_list, crs=2056)
h3_500_gdf['index'] = h3_500_gdf.index

In [ ]:
h3_500_GM = gpd.sjoin(h3_500_gdf, data_final[['uuid','NOANNEE','geometry']], predicate = 'intersects')
h3_500_GM[['index','uuid']] = h3_500_GM[['index','uuid']].astype('category')
h3_500_GM = h3_500_GM.groupby(['index','NOANNEE']).uuid.nunique().reset_index()
h3_500_GM_merged = pd.merge(h3_500_gdf, h3_500_GM, on = ['index'])

In [ ]:
h3_500_GM_merged.columns = ['geometry','h3_id','NOANNEE','n']

In [ ]:
ax = h3_500_GM_merged[h3_500_GM_merged.NOANNEE == 2017].plot('n', legend =True, figsize = (20,20), legend_kwds = {'shrink':0.3})
ax.set_axis_off()

### Calculate averages by h3

In [ ]:
joined_gdf = gpd.sjoin(data_final,h3_500_gdf[['geometry']], predicate = 'intersects')

In [ ]:
joined_gdf.drop(['uuid','CDPHYSSEXE','age_group','CANTON_NAME','CANTON_ACRONYM','geometry'], axis = 1).groupby(['index_right','NOANNEE']).mean()

In [ ]:
agg_data_final = joined_gdf.drop(['uuid','CDPHYSSEXE','age_group','CANTON_NAME','CANTON_ACRONYM','geometry'], axis = 1).groupby(['index_right','NOANNEE']).mean().reset_index()

In [ ]:
agg_data_final = agg_data_final.rename(columns = {'index_right':'h3_id'})

In [ ]:
h3_500_GM_merged_wfeatures = pd.merge(h3_500_GM_merged, agg_data_final, on = ['h3_id','NOANNEE'], how = 'left')

### Global autocorrelation of main features - Moran's I

In [ ]:
from libpysal.weights import Queen, Rook, KNN
from libpysal.weights import lat2W, higher_order

In [ ]:
agg_results_folder = results_folder/'Aggregated'

In [ ]:
globalautocorr_agg_result_folder = agg_results_folder/'Global Autocorrelation'
if not os.path.exists(globalautocorr_agg_result_folder):
    os.makedirs(globalautocorr_agg_result_folder)

In [ ]:
localautocorr_agg_result_folder = agg_results_folder/'Local Autocorrelation'
if not os.path.exists(localautocorr_agg_result_folder):
    os.makedirs(localautocorr_agg_result_folder)

In [ ]:
weights_by_year_h3 = {}
weights_by_year_h3_pop_scaled = {}

In [ ]:
def get_weights(df, nn, pop_scaled=False):
    print('Calculating weights')
    w = lps.weights.KNN(cKDTree(get_points_array(df.geometry.centroid)), nn)
    if pop_scaled:
        print('Scaling weights')
        # Adjust weights by the square root of the product of populations
        for i, neighbors in w.neighbors.items():
            for j in neighbors:
                original_weight = w.weights[i][neighbors.index(j)]
                adjusted_weight = original_weight *  df['n'].iloc[j]
                w.weights[i][neighbors.index(j)] = adjusted_weight
                w.transform = 'R'
        return w
    else:
        return w

In [ ]:
col_names = ['PRESTATIONS_BRUTES_AOS', 'PRESTATIONS_BRUTES_LCA', 'cds']  # replace with your actual column names
# Calculate weights once for each year and store them
years = [2017, 2018, 2019, 2020, 2021]
nn = 18

for col_name in col_names:
    if col_name not in weights_by_year_h3.keys():
        weights_by_year_h3[col_name] = {}
        weights_by_year_h3_pop_scaled[col_name] = {}
    for year in years:
        print(col_name, year)
        # Store the weights in the dictionary
        df = h3_500_GM_merged_wfeatures[h3_500_GM_merged_wfeatures.NOANNEE == year]
        df_nonull = df[df[col_name].isnull()==False]
        if year not in weights_by_year_h3[col_name].keys():
            weights_by_year_h3[col_name][year] = get_weights(df_nonull, nn, pop_scaled=False)
            weights_by_year_h3_pop_scaled[col_name][year] = get_weights(df_nonull, nn, pop_scaled=True) 


# Now iterate over each column and use the pre-calculated weights
for col_name in col_names:
    for year in years:
        df = h3_500_GM_merged_wfeatures[h3_500_GM_merged_wfeatures.NOANNEE == year]
        df_nonull = df[df[col_name].isnull()==False]
        w = weights_by_year_h3[col_name][year]  # Retrieve pre-calculated weights
        mi = GlobalMoranI(db=df_nonull, col=col_name, year=year, distance=nn, w=w, result_folder=globalautocorr_agg_result_folder)

In [ ]:
year = 2018
col_name = 'PRESTATIONS_BRUTES_AOS'
for nn in [6,18,36, 72, 144, 288]:
    df = h3_500_GM_merged_wfeatures[h3_500_GM_merged_wfeatures.NOANNEE == year]
    df_nonull = df[df[col_name].isnull()==False]
    w = lps.weights.KNN(cKDTree(get_points_array(df_nonull.geometry.centroid)), nn)  # Retrieve pre-calculated weights
    mi = GlobalMoranI(db=df_nonull, col=col_name, year=year, distance=nn, w=w, result_folder=globalautocorr_agg_result_folder)

### Local autocorrelation with Getis Ord Gi

In [ ]:
from esda import fdr

In [ ]:
nn = 36

In [ ]:
def compute_plot_getis(df_merged, col_names, years, nn, pop_scaled, result_folder):
    df_getis = {}
    for col_name in col_names:
        df_getis[col_name] = {}
        for year in years:
            df = df_merged[df_merged.NOANNEE == year]
            df_nonull = df[df[col_name].notnull()]

            if pop_scaled:
                w = get_weights(df_nonull, nn, pop_scaled=True)
                getis_values = pyspace.compute_getis(df_nonull, col_name, w, 9999, star=False, transform_type='R', p_001=True)

            else:
                w = get_weights(df_nonull, nn)
                getis_values = pyspace.compute_getis(df_nonull, col_name, w, 9999, star=False, transform_type='B', p_001=True)
            xlabel = f"Getis - {col_name} - {year} - {nn}NN"
            file_path = Path(result_folder) / f'{xlabel}.png'
            if not file_path.exists():
                fig, ax = pyspace.plotGetisMap(df_nonull, f"{col_name}_G_cl", markersize_s=0.01, markersize_l=0.1, p_001=True, commune_name=False)
                plt.savefig(file_path, dpi=1000, bbox_inches='tight')
            
            # FDR adjustment
            fdr_pvalue = fdr(getis_values.p_sim, 0.05)
            df_nonull[f"{col_name}_G_cl_fdr"] = df_nonull[f"{col_name}_G_cl"]
            df_nonull.loc[df_nonull[f"{col_name}_G_psim"] >= fdr_pvalue, f"{col_name}_G_cl_fdr"] = 'Not significant'
            
            xlabel_fdr = f"Getis - {col_name} - {year} - {nn}NN_fdr"
            file_path_fdr = Path(result_folder) / f'{xlabel_fdr}.png'
            if not file_path_fdr.exists():
                fig, ax = pyspace.plotGetisMap(df_nonull, f"{col_name}_G_cl_fdr", markersize_s=0.01, markersize_l=0.1, p_001=True, commune_name=False)
                plt.savefig(file_path_fdr, dpi=1000, bbox_inches='tight')
            
            df_getis[col_name][year] = df_nonull
    
    return df_getis

In [ ]:
# Binary weighting matrix
df_getis_results = compute_plot_getis(h3_500_GM_merged_wfeatures, col_names, years, nn, False, localautocorr_agg_result_folder)

In [ ]:
# Population scaled weights (more populated h3 have more weights) then row-standardized
df_getis_results_pop_scaled = compute_plot_getis(h3_500_GM_merged_wfeatures, col_names, years, nn, True, localautocorr_agg_result_folder/'Population Scaled weights')

In [ ]:
dict_labels = {'PRESTATIONS_BRUTES_AOS':'Conv. Med. Yearly Spending (CHF)',
              'PRESTATIONS_BRUTES_LCA':'Compl. Med. Yearly Spending (CHF)',
              'cds':'Chronic Disease Score (CDS)',
              'ihs_cost_lca':'IHS transformed CAM claims amount'}

### Avg by Getis class - Bar plots

In [ ]:
for col_name in col_names:
    for year in years:
        xlabel = f"Getis Bar Plot - {col_name} - {year} - {nn}NN"
        file_path = Path(localautocorr_agg_result_folder) / f'{xlabel}.png'

        fig, ax = pyspace.plot_getis_by_class(df_getis_results[col_name][year],f'{col_name}_G_cl', col_name, dict_labels[col_name], 8, 12, 8, 8, p_001=True, showfliers = False)
        plt.savefig(file_path, dpi=320, bbox_inches='tight')

In [ ]:
for col_name in col_names:
    for year in years:
        xlabel = f"Getis Bar Plot - {col_name} - {year} - {nn}NN"
        file_path = Path(localautocorr_agg_result_folder/'Population Scaled weights') / f'{xlabel}.png'

        fig, ax = pyspace.plot_getis_by_class(df_getis_results_pop_scaled[col_name][year],f'{col_name}_G_cl', col_name, dict_labels[col_name], 8, 12, 8, 8, p_001=True, showfliers = False)
        plt.savefig(file_path, dpi=320, bbox_inches='tight')

## Adjustment models

### 1. OLS

In [ ]:
from spreg import OLS
from spreg import ML_Lag
from spreg import GM_Lag
from spreg import ML_Error
from spreg import GM_Error
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
df_test = df_getis_results['PRESTATIONS_BRUTES_LCA'][year]
w_test = weights_by_year_h3['PRESTATIONS_BRUTES_LCA'][year]
y = 'PRESTATIONS_BRUTES_LCA'
X = ['NBAGE','ssep2','SEX_F','cds']
df_test[X] = StandardScaler().fit_transform(df_test[X])

In [ ]:
def run_ols(df,X,y,w):
    g_y = df[y].values.reshape(-1, 1)
    g_X = df[X].values
    ols = OLS(g_y, g_X, w, spat_diag=True, moran=True, name_y=y, name_x=X)
    return ols

In [ ]:
ols = run_ols(df_test, X, y, w_test)

In [ ]:
print(ols.summary)

In [ ]:
df_test['OLS_residuals'] = ols.u

### Spatial error model

In [ ]:
def run_mlerr(df,X,y,w):
    g_y = df[y].values.reshape(-1, 1)
    g_X = df[X].values
    mlerr = ML_Error(g_y,g_X,w,name_y=y,name_x=X,name_w='36NN') 
    return mlerr

In [ ]:
mlerr = run_mlerr(df_test, X, y, w_test)

In [ ]:
print(mlerr.summary)

In [ ]:
df_test['MLERR_residuals'] = mlerr.u

### GWR

In [ ]:
import mgwr
reload(mgwr)

In [ ]:
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap,compare_surfaces
from shapely.geometry import Point
import multiprocessing as mp
from contextlib import redirect_stdout
import itertools

In [ ]:
def model_prep(df,y,X):
    """Prepare data for subsequent GWR/MGWR"""
    g_y = df[y].values.reshape(-1, 1)
    g_X = df[X].values
    u = df['E']
    v = df['N']
    g_coords = list(zip(u,v))
    g_X = (g_X - g_X.mean(axis=0)) / g_X.std(axis=0)
    g_y = (g_y - g_y.mean(axis=0)) / g_y.std(axis=0)
    return g_y, g_X, g_coords

In [ ]:
df_test['E'] = df_test['geometry'].centroid.x
df_test['N'] = df_test['geometry'].centroid.y

In [ ]:
df_test.to_file('df_test_mgwr.shp')

In [ ]:
n_proc = 9
g_y,g_X,g_coords = model_prep(df_test,y,X)
pool = mp.Pool(n_proc)
gwr_selector = Sel_BW(g_coords, g_y, g_X, fixed=False, kernel='bisquare')
gwr_bw = gwr_selector.search(verbose = True,criterion='AICc',interval = 100,pool = pool)
pool.close()

In [ ]:
model = GWR(g_coords, g_y, g_X, bw=gwr_bw, fixed=False, kernel='bisquare')
pool = mp.Pool(n_proc)
gwr_results = model.fit(pool = pool)
pool.close()

In [ ]:
gwr_results_filtered = gwr_results.filter_tvals()

In [ ]:
df_test['GWR_residuals'] = gwr_results.resid_response

### MGWR

In [ ]:
mgwr_agg_result_folder = agg_results_folder/'GWR_MGWR'
if not os.path.exists(mgwr_agg_result_folder):
    os.makedirs(mgwr_agg_result_folder)

In [ ]:
y = (y - y.mean(axis=0)) / y.std(axis=0)

In [ ]:

_data = lps.io.open(lps.examples.get_path('GData_utm.csv'))
_coords = list(zip(_data.by_col('X'), _data.by_col('Y')))
_y = np.array(_data.by_col('PctBach')).reshape((-1,1))

In [ ]:
g_y,g_X,g_coords = model_prep(df_test,y,X)
pool = mp.Pool(n_proc)
mgwr_selector = Sel_BW(g_coords, g_y, g_X, fixed=False,multi = True, kernel='bisquare')
mgwr_bw = mgwr_selector.search(verbose = True,criterion='AICc',pool = pool)
pool.close()


In [ ]:
pool.close()

In [ ]:
model = MGWR(g_coords, g_y, g_X, selector=mgwr_selector, fixed=False, kernel='bisquare')
pool = mp.Pool(n_proc)
mgwr_results = model.fit(pool = pool)
pool.close()

In [ ]:
mgwr_results_filtered = mgwr_results.filter_tvals()

In [ ]:
df_test['MGWR_residuals'] = mgwr_results.resid_response

In [ ]:
sum(df_test['MGWR_residuals']**2)

In [ ]:
sum(df_test['GWR_residuals']**2)

In [ ]:
sum(df_test['MLERR_residuals']**2)

In [ ]:
sum(df_test['OLS_residuals']**2)

In [ ]:
reload(mgwr.utils)
def plot_gwr_mgwr(df,X,gwr_bw,mgwr_bw,gwr_results,mgwr_results,gwr_filtered_t,mgwr_filtered_t,result_dir):
    kwargs1 = {'edgecolor': 'lightgrey', 'markersize': 100}
    kwargs2 = {'facecolor': 'lightgrey', 'markersize': 100}
#     df['gwr_intercept'] = gwr_results.params[:, 0]
#     df['mgwr_intercept'] = mgwr_results.params[:, 0]
    if 'intercept' not in X:
        X.insert(0,'intercept')
    for i,var in enumerate(X):
        print(i,var)
        var_mgwr = 'mgwr_'+var
        var_gwr = 'gwr_'+var
        df[var_gwr] = gwr_results.params[:, i]
        df[var_mgwr] = mgwr_results.params[:, i]

    print('*'*30,'Start of plotting step','*'*30)
    for i,var in enumerate(X):
        var_gwr,var_mgwr = 'gwr_'+ var,'mgwr_'+ var
        mgwr.utils.compare_surfaces(df, var_gwr, var_mgwr,
        gwr_filtered_t[:, i], gwr_bw,
        mgwr_filtered_t[:, i], mgwr_bw[i],
        var, kwargs1, kwargs2, savefig = result_dir/var)

In [ ]:
plot_gwr_mgwr(df_test, X, gwr_bw, mgwr_bw, gwr_results, mgwr_results, gwr_results_filtered, mgwr_results_filtered, mgwr_agg_result_folder)

In [ ]:
import matplotlib as mpl
mpl.colormaps['viridis']